In [217]:
import pandas as pd 
import spotipy
import numpy as np
from spotipy.oauth2 import SpotifyOAuth 
import pickle
from spotipy.oauth2 import SpotifyClientCredentials
scope1 = "user-library-read, user-library-modify, playlist-modify-public, playlist-modify-private, user-library-modify, playlist-read-collaborative, app-remote-control"
client_id = "15fd015d4da742d0bf7da9c6ed3120c7"
client_secret = "349c5181636e4e3cbb974b051db425fd"
sp =spotipy.Spotify(auth_manager= SpotifyOAuth(client_id = "15fd015d4da742d0bf7da9c6ed3120c7", client_secret = "349c5181636e4e3cbb974b051db425fd", redirect_uri = "http://example.com", scope= scope1))

In [218]:
filename = 'model.pkl'
with open(filename, 'rb') as file:
    loaded_model = pickle.load(file)

In [219]:
#create playlist
name = ["Calm", "Happy", "Energy", "Sad"]
for i in range(len(name)):
    playlist_name = name[i]
    user_id = sp.me()["id"]
    sp.user_playlist_create(user=user_id, name=playlist_name, public=True)

In [220]:
#load all the liked song from users
liked_songs = []
results = sp.current_user_saved_tracks()
while results:
    for item in results['items']:
        track = item['track']
        song_id = track['id']
        row = {"song_id": song_id}
        liked_songs.append(row)
    results = sp.next(results)
output = pd.DataFrame(liked_songs)

In [221]:
#gather data using soptify API
song_ana = pd.DataFrame()
for i in range (len(output)) :
    analysis = sp.audio_features(output.at[i, "song_id"])
    analy = pd.DataFrame(analysis)
    song_ana = pd.concat([song_ana, analy])
song_ana = song_ana.drop(["type", "duration_ms", "time_signature", "mode"], axis= 1).reset_index().drop(["index"], axis= 1)

In [222]:
# Normalize the DataFrame to be between 0 and 1
min_value = song_ana[["tempo"]].min()
max_value = song_ana[["tempo"]].max()
song_ana["tempo"] = (song_ana[["tempo"]] - min_value) / (max_value - min_value)

min_value = song_ana[["loudness"]].min()
max_value = song_ana[["loudness"]].max()
song_ana["loudness"] = (song_ana[["loudness"]] - min_value) / (max_value - min_value)

min_value = song_ana[["key"]].min()
max_value = song_ana[["key"]].max()
song_ana["key"] = (song_ana[["key"]] - min_value) / (max_value - min_value)


*starting to classifed*

In [223]:
#target cloumn
column_select = song_ana[["danceability", "energy", "key", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo"]]

In [224]:
song_ana["prediciton"] = loaded_model.predict(column_select)

In [225]:
calm = song_ana[song_ana['prediciton'] == 0 ].reset_index()
happy = song_ana[song_ana['prediciton'] == 1 ].reset_index()
energy = song_ana[song_ana['prediciton'] == 2 ].reset_index()
sad = song_ana[song_ana['prediciton'] == 3 ].reset_index()

In [226]:
print(len(calm))
print(len(happy))
print(len(energy))
print(len(sad))

42
90
595
948


In [245]:
playlist_array = [calm, happy, energy, sad]
name = ["Calm", "Happy", "Energy", "Sad"]

In [242]:
# getting playlist id
for i in range(len(name)):
    playlist = pd.DataFrame(sp.current_user_playlists()["items"])
    find_id = playlist[playlist["name"] == name[i]].reset_index()
    playlist_id = find_id.at[0,"id"]
    # add song to the playlist
    for j in range(len(playlist_array[i])):
        sp.playlist_add_items(playlist_id = playlist_id, items = [playlist_array[i].at[j,"id"]])